# INTRODUÇÃO
Uma questão ao procuramos dados do mercado financeiro é a fonte de dados, pois podem divergir.

A B3 fornece dados públicos que podem ser acessado no site.

# OBJETIVO
Construir, a partir dos dados públicos fornecidos pela B3, um "_Data Frame_" com as colunas OHLCV (Open - Abertura, High - Máxima, Low - Mínima, Close - Fechamento e Volume) em uma frequência temporal.  
Deixando pronto para construção de gráficos de Velas, candlesticks.

# Instalando e Importando Bibliotecas
Utilizaremos a biblioteca ["_Pendulum_"](https://pendulum.eustace.io/) para tratamento de datas e horas e a ["_Pandas_"](https://pandas.pydata.org/) para a importação e tratamento dos dados.

In [1]:
!pip install pendulum

In [2]:
import pandas as pd
import pendulum as pl

# Importando os Dados
Este arquivo é disponibilizado no site da [B3, Dados Públicos, Negócio a Negócio - Listados](https://www.b3.com.br/pt_br/market-data-e-indices/servicos-de-dados/market-data/consultas/boletim-diario/dados-publicos-de-produtos-listados-e-de-balcao/). Por ser grande, recomendo importar os dados para o computador local, realizando o tratamento com mais eficiência.

In [4]:
df = pd.read_csv('TradeIntraday_20220503_1.txt', sep=';', dtype=str)

# O Ativo Trabalhado
Vamos trabalhar com o contrato do Mini Dólar Futuro.

In [10]:
wdom22 = df[df['TckrSymb'] == 'WDOM22']
wdom22.drop(wdom22.columns[[1, 2, 7, 8]], axis = 1, inplace = True)
wdom22 = wdom22.reset_index(drop=True)

# Data e Horas
Organizamos a data e o tempo.

In [12]:
wdom22['H_M_S'] = wdom22['NtryTm'].str[0:2] + ':' + wdom22['NtryTm'].str[2:4] + ':' + wdom22['NtryTm'].str[4:6] + '.' + wdom22['NtryTm'].str[6:]
wdom22['date_time'] = wdom22['RptDt'] + 'T' + wdom22['H_M_S']
wdom22['date_time'] = [pl.parse(i) for i in wdom22['date_time']]

# Organizando os Dados
Transformamos a data e o tempo em indice.  
Selecionamos as colunas de interesse e renomeamos.

In [20]:
wdom22_1 = wdom22.set_index('date_time')
wdom22_1.drop(wdom22.columns[[0, 3, 5]], axis = 1, inplace = True)
wdom22_1.rename(columns = {'GrssTradAmt': 'Preco', 'TradQty': 'Quantidade', 'TradId': 'Negocio'}, inplace = True)

# Transformando os Dados
Operações matemática serão necessárias, transformamos os dados para aceitar os calculos.

In [31]:
wdom22_1['Preco'] = wdom22_1['Preco'].str.replace(',', '.').astype(float)
wdom22_1['Quantidade'] = wdom22_1['Quantidade'].astype(int)
wdom22_1['Negocio'] = wdom22_1['Negocio'].astype(int)
wdom22_1['Negocio'] = (wdom22_1['Negocio'] / 10).astype(int)

# Definindo o Tempo
Difinimos o tempo gráfico que queremos a tabela OHLCV.

In [36]:
t = '5T'

# Construindo o Data Frame

In [38]:
wdom22_2 = pd.DataFrame({'Open': wdom22_1['Preco'].resample(t).first(), 
             'High': wdom22_1['Preco'].resample(t).max(),
             'Low': wdom22_1['Preco'].resample(t).min(),
             'Close': wdom22_1['Preco'].resample(t).last(),
             'Volume': wdom22_1['Quantidade'].resample(t).sum()})

# Visualizando a Tabela dos Dados

In [40]:
wdom22_2

,Open,High,Low,Close,Volume
date_time,,,,,
2022-05-03 09:00:00+00:00,5085.0,5096.0,5078.5,5092.5,98155
2022-05-03 09:05:00+00:00,5093.0,5095.5,5075.5,5083.5,74906
2022-05-03 09:10:00+00:00,5083.0,5087.0,5075.5,5078.0,65573
2022-05-03 09:15:00+00:00,5078.0,5084.5,5073.0,5082.0,63925
2022-05-03 09:20:00+00:00,5082.0,5092.0,5080.0,5087.0,55502
...,...,...,...,...,...
2022-05-03 17:35:00+00:00,5011.5,5012.0,5008.5,5009.5,7169
2022-05-03 17:40:00+00:00,5009.5,5013.0,5008.5,5011.5,10098
2022-05-03 17:45:00+00:00,5011.5,5012.5,5008.0,5008.5,14046


# Salvando
Salvamos a nova tabela como um arquivo "_.csv_".

In [42]:
#wdom22_2.to_csv('2022_05_03_wdom22.csv', index = True, encoding = 'UTF-8')